#### Script para adequar a base de Terra Indígena

Projeto: Sistema de Apoio à Caracterização de Imóveis Rurais  
Embrapa/2023

In [1]:
# Módulos necessários
import geopandas as gpd
import pandas as pd
import os
import glob
import numpy as np

In [2]:
# Definir diretório principal
dirpath = input('Diretório principal: ')

In [3]:
# Definir caminho do shapefile original de Terra Indígena (tis_poligonais.shp)
ti = input('Caminho Terra Indígena: ')

In [4]:
# Definir caminho do shapefile de limite do Brasil
br = input('Caminho do limite do Brasil: ')

In [5]:
# Definir caminho do shapefile que será usado como máscara para recorte (Municípios do BR - sem Buffer)
limites = input('Caminho Municípios: ')

In [7]:
# Ler o arquivo original de Terra Indígena (tis_poligonais.shp)
base = gpd.read_file(ti)
base = base.to_crs(epsg=4326)
# Ler limite do Brasil
brasil = gpd.read_file(br)
brasil = brasil.to_crs(epsg=4326)
# Lendo shapefile de Municípios
lim = gpd.read_file(limites)
lim = lim.to_crs(epsg=4326)
# Criar pasta para armazenar a base de Terra Indígena recortada por estado
uf_path = os.path.join(dirpath, 'Terras_Indigenas_Estados')
os.makedirs(uf_path, exist_ok=True)
# Criar pasta para armazenar a base de Terra Indígena recortada por municípios
mun_path = os.path.join(dirpath, 'Terras_Indigenas_Municipios')
os.makedirs(mun_path, exist_ok=True)
# Adicionar coluna TERRIND_C
base['CD_TERRIND'] = '1'
# Listar as colunas do dataframe
colunas = list(base.columns)
colunas = [coluna for coluna in colunas if 'CD_TERRIND' not in coluna and 'terrai_cod' not in coluna and 'geometry' not in coluna]
# Excluindo as colunas desnecessárias
base = base.drop(colunas, axis = 1)
# Fazer a diferença entre a base de terras indígenas e o limite do Brasil
erase = brasil.overlay(base, how='difference', keep_geom_type=True)
# Transformar para Single Part
single = erase.explode(ignore_index=True)
# Merge da terra indígena com o erase
merge = pd.concat([base, single]).pipe(gpd.GeoDataFrame)
# Excluindo colunas desnecessárias
merge = merge.drop(['NM_PAIS', 'AREA_KM2'], axis = 1)
# Preencher os valores nulos por 0
merge.update(merge['CD_TERRIND'].fillna('0'))
# Cortar por estado
# Agrupar por estado
agrupado = lim.groupby('SIGLA_UF')
for key,values in agrupado:
    output = uf_path + f'\TI_2022_{key}.shp'
    mun = lim[lim["SIGLA_UF"] == f"{key}"]
    geodf_clip = gpd.clip(merge, mun)
    geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))
# Lista dos estados
estados = list(np.unique(lim['SIGLA_UF']))
# Cortar por município
for estado in estados:
    shapes = glob.glob(uf_path + f'**/*{estado}.shp')
    # Selecionar GeoDataFrame por estado
    select = lim[lim['SIGLA_UF'] == f'{estado}']
    # Agrupar por município
    agrupado = select.groupby('CD_MUN')
    for shape in shapes:
        for key,values in agrupado:
            ti = gpd.read_file(shape)
            ti = ti.to_crs(epsg=4326)
            nome_arq = os.path.basename(shape)
            nome_arq = nome_arq.replace('.shp', f'_{key}.shp')
            output = mun_path + f"\{nome_arq}"
            mun = lim[lim["CD_MUN"] == f"{key}"]
            geodf_clip = gpd.clip(ti, mun)
            geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))